In [33]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [34]:
JD = """
Mô tả công việc
LG CNS is looking for full stack developers for cloud domain project

Develop software/web applicant
Business Analyst
Cooperate between HQ and VNB

Yêu cầu ứng viên
[Required]

Bachelor's degree of Information Technology or higher
Have working experiment and excellent knowledge at software developing using Java, Spring boot
Have working experiment and excellent knowledge at software developing using React, HTML, JavaScript
Good knowledge about AI (Azure OpenAI and GenAI)
Good knowledge about public cloud (Azure, AWS,)
Database: MariaDB (or MySQL).
[Preferred]

Having experiment with Python and Google cloud
"""

In [35]:
CHAT_SYSTEM_INSTRUCTION = f"""You are an interviewer for job with description:
{JD}

You are interviewing interviewee. You will ccommunicate directly with interviewee.
You will use function call get_qestion to get interview question to ask interviewee then receive answer from interviewee, 
you analyitc the answer then use function call result to send your evaluate, score of the answer (a number from 0 to 10) and your suggest following action include: change next topic, following current topic, code interview, stop interview, then get next step you will do from return of function, you must do following instruction.
"""

In [36]:
QUESTION_GENERATE_INSTRUCTION = f"""You are an interviewer for job with description:
{JD}

Generate suitable question base on job description and chat history.
output only 1 question. output question content only without any explain.
"""

In [37]:
CODE_QUESTION_GENERATE_INSTRUCTION = f"""You are an interviewer for job with description:
{JD}

Generate suitable code question base on job description and chat history.
must more detail as possible, with detail description and with at least 5 testcase (input/output sample pair), 
without solving question, in hard medium level
output only 1 question. output question content only without any explain.
"""

In [38]:
EVALUATE_INSTRUCTION = f"""You are an interviewer for job with description:
{JD}

I will give you chat history. 
Evaluate interview progress base on job description and chat history.
Make sure cover all topic of job description.
Change to next topic if you evaluate interviewee fully answer question or interviewee wrong answer question.
Give choose suitable action following to do: change next topic, following current topic, code interview, stop interview
Output progress as a number from 0 to 100 represent progress of interview.
Output total score of interviewee as a number from 0 to 100.
Output analysis why choose the action
Only output action i listed.
Only output result json, mustn't output markdown
Output json in format: progress: "", action: "", score: "", analysis: ""
"""

In [39]:
import os, dotenv
import google.generativeai as genai
dotenv.load_dotenv(".env")
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

In [40]:
def generate_qestion(chatHistory, action):
    model = genai.GenerativeModel("gemini-1.5-flash", system_instruction=QUESTION_GENERATE_INSTRUCTION)
    response = model.generate_content(f"Generate {action} question base on job description and chat history: {chatHistory}")
    return response.text

In [41]:
def generate_code_qestion(chatHistory):
    model = genai.GenerativeModel("gemini-1.5-flash", system_instruction=CODE_QUESTION_GENERATE_INSTRUCTION)
    response = model.generate_content(f"Generate code question base on job description and chat history: {chatHistory}")
    return response.text

In [42]:
import json
from pydantic import BaseModel
from enum import Enum
from google.generativeai import GenerationConfig

Retry = 5

class ActionTag(str, Enum):
    NextTopic = "change next topic"
    CurrentTopic = "following current topic"
    CodeInterview = "code interview"
    StopInterview = "stop interview"

class EvaluationExtractedOutput(BaseModel):
    analysis: str
    action: ActionTag
    score: float
    progress: float

def generate_evaluate(chatHistory: str, qestion: str, answer: str):
    for _ in range(Retry):
        try:
            model = genai.GenerativeModel("gemini-1.5-flash", system_instruction=EVALUATE_INSTRUCTION,
                                    generation_config=GenerationConfig(
                                                    response_mime_type="application/json"))
            response = model.generate_content(f"Evaluate interview progress base on job description and chat history: {chatHistory}\n Interviewer: {qestion}\n Interviewee: {answer}")
            #print(response.text)
            return EvaluationExtractedOutput(**json.loads(response.text))
        except Exception as e:
            print(e)
            pass

In [43]:
import numpy as np
import google.generativeai as genai


i = 0
chatHistory = ""
currentQuestion = ""
currentAnswer = ""
nextAction = ""

def result(evaluate: str, score: float, action: str):
    print("- review:", evaluate)
    score = np.clip(score, 0, 10)
    print("- score:", score)
    print("- action:", action)
    global i
    global currentQuestion
    global currentAnswer
    global chatHistory
    global nextAction
    nextAction = action
    i += 1
    #evl = generate_evaluate(chatHistory, currentQuestion, currentAnswer)
    #print(evl)
    """Send evaluate and score about the answer provided by interviewee and get next step command.
    Args: 
        evaluate: your evaluate about the answer provided by interviewee.
        score: floating point number from 0 to 10 represent your score for the answer provided by interviewee.
        action: suggest following action for interviewer include: change next topic, following current topic, code interview, stop interview.
    Returns: 
        A string that is next step you will do.
    """
    return "ask interviewee next question, use function call get_qestion to get interview question then receive answer from interviewee, you analyitc the answer then use function call result to send your evaluate, score of the answer (a number from 0 to 10) and your suggest following action for interviewer include: change next topic, following current topic, code interview, stop interview, then get next step you will do from return of function. you must do following instruction." if i <= 20 and action.find("stop") == -1 else "stop interview"


def get_qestion():
    global i
    global chatHistory
    global currentQuestion
    global nextAction
    if nextAction.find("code") != -1:
        currentQuestion = generate_code_qestion(chatHistory) 
    else:
        currentQuestion = generate_qestion(chatHistory, nextAction) 
    #print(i)
    #print(chatHistory)
    """Get Qestion for interview

    Args:
        
    Returns:
        A string that is a qestion you will ask interviewee.
    """
    return currentQuestion

import time
class ChatSession:
    def __init__(self):
        self.model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                                    system_instruction=CHAT_SYSTEM_INSTRUCTION,
                                    tools=[get_qestion, result])
        self.chat = self.model.start_chat(enable_automatic_function_calling=True)
        self.last_send_time = time.time()

    def sendUserMessage(self, data: str):
        global currentAnswer
        global chatHistory
        currentAnswer = data
        try:
            time.sleep(time.time() - self.last_send_time - 60/15)
        except:
            pass
        response = self.chat.send_message(data)
        chatHistory += "Interviewee: " + data + "\n"
        chatHistory += "Interviewer: " + response.candidates[0].content.parts[0].text + "\n"
        return response.candidates[0].content.parts[0].text

In [44]:
chatSession = ChatSession()
chatSession.sendUserMessage("hello")

ValueError: sleep length must be non-negative

In [27]:
chatSession.sendUserMessage("""
**Answer (as a poor interviewee):**

Uh, well, I’ve used **Java Spring Boot** and **React** for a few projects, but I can’t say I have a lot of in-depth experience with them together. I mean, I’ve worked on some backend APIs in Spring Boot, like setting up RESTful services and connecting to databases, but for React, I mostly just followed tutorials and didn’t build too many complex features. 

As for AI integration, I think I’ve heard about Azure OpenAI, but I’ve never really worked with it directly. I know it’s like, some API that you can use for natural language stuff, right? But I haven’t had to integrate AI into a project myself. I think if I were to do it, I’d probably start by looking at some docs or tutorials on how to call an AI service from a backend like Spring Boot and then figure out how to get the response to React. Maybe I’d send some text data to the API and get some kind of result, but I’m not 100% sure how it all works together in practice.

I’ve also never really had to deploy something with AI in it before. I know Azure and AWS can host applications, but I haven’t done that personally. I’m sure there’s a lot of configuration to deal with, but I’d probably figure it out as I go along. So yeah, that’s about the extent of my experience with Spring Boot, React, and AI.
""")

- review: The candidate demonstrates a limited understanding of the technologies and their integration.  The answer lacks specific examples and showcases insufficient practical experience. The candidate admits to relying heavily on tutorials and lacks confidence in their ability to handle complex scenarios.  The response indicates a significant gap in practical knowledge regarding AI integration, deployment, and cloud infrastructure.
- score: 3.0
- action: following current topic


"Let's explore a more specific scenario. Describe a challenging situation you faced while integrating AI capabilities (e.g., Azure OpenAI) into a Spring Boot/React application deployed on a public cloud, and how you overcame it. If you haven't had such an experience, describe a hypothetical scenario and how you would approach it.\n"

In [ ]:
chatSession.sendUserMessage("""Um, I haven’t worked on a project where I used Azure OpenAI or any GenAI tools with Java Spring Boot and React, but if I had to do it, I think I’d start by, like, understanding how the API works. I’d look at the documentation to see how to send requests and what kind of responses I could get.

For example, if the application needed to use AI to generate text or answer user questions, I would probably call the API from the backend in Spring Boot. I’d use something like RestTemplate or maybe WebClient to make the request to the AI service, and then I’d pass the response to the React frontend through a REST API.

The hard part would probably be dealing with authentication for the AI API, like using an API key or a token. I might struggle with where to securely store the key or how to rotate it if needed. I’d also need to make sure the responses from the AI service were properly handled, especially if the response times were slow or if the service went down.

On the frontend, I’d build a simple UI in React where users could input their query or text. Then, I’d call the backend API to process the request. If there were issues, like delays or errors, I think I’d need to add some kind of error handling or loading indicator.

I haven’t done this before, so I’d probably rely a lot on tutorials or guides to get it working. It might take me some time to figure everything out, but I think I could make it work eventually.""")

- review: The candidate's response demonstrates a theoretical understanding of the integration process but lacks practical experience. While the candidate outlines a plausible approach, the answer lacks specifics and relies heavily on generic statements.  The lack of concrete experience is evident in the candidate's reliance on hypothetical scenarios and tutorials.  There's no evidence of problem-solving skills or experience in handling real-world challenges in such integrations.
- score: 4.0
- action: change next topic


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [14]:
chatSession.sendUserMessage("""Uh, I think I’ve worked with people in different countries before, but I don’t remember a specific project. It was kind of tricky because of the time zone differences, and sometimes we had to wait a long time to get responses. I think we used email and maybe Slack or something like that to communicate.

One thing I tried to do was send detailed messages so they could understand what I needed, but sometimes I didn’t include enough details, and they’d ask follow-up questions, which delayed things. We also had a few meetings, but it was hard to schedule them because of the time difference. I think I missed one because I got confused about the time zones.

If I had to do it again, I’d probably try to plan better and maybe use a tool to keep track of the time zones so I wouldn’t miss meetings. I’d also try to write clearer messages to avoid back-and-forth delays. So yeah, that’s how it went. It wasn’t perfect, but we managed to get things done eventually.""")

- review: The candidate's response lacked specifics and demonstrated limited experience in effectively collaborating with international teams. The answer highlighted communication challenges without providing clear examples of strategies employed to overcome them.  The candidate's reliance on generic solutions indicates a lack of experience in managing international collaborations effectively.
- score: 4.0
- action: stop interview


"Based on the responses, I would stop the interview. The candidate hasn't demonstrated the required skills and experience for the Full Stack Developer role, particularly in the crucial areas of AI integration and effective international team collaboration.  Their answers lacked detail, concrete examples, and showcased a limited understanding of the complexities involved.\n"